In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import os
import pandas as pd
from Priorizacion_funciones.ocupacion_nodos_sin_exo import *
from Priorizacion_funciones.modelo_forecast_sin_exo import *
from Priorizacion_funciones.series_generadas import *
import warnings
warnings.filterwarnings('ignore')
os.getcwd()

'C:\\Users\\CristhianJohnatanIzq\\OneDrive - SQDM\\Proyecto2\\ENVIO_DARIO\\MODELO_PRIORIZACION_DATA_ORIGEN_ADRIANA'

***Dejar sólo un archivo de ocupación completo: si lo envían completo reemplazar el archivo en la carpeta ocupación, si envian sólo el último mes se debe generar sólo uno.***


In [3]:
data_completa = pd.read_csv('ocupacion/OCUPACION_COMPLETO.csv')
data_completa['MES'] = [f'0{x}' if x <10 else str(x) for x in data_completa['MES']]
data_completa['DIA'] = [f'0{x}' if x <10 else str(x) for x in data_completa['DIA']]
data_completa['FECHA'] = data_completa['ANIO'].astype('str') + '-' + data_completa['MES'] + '-' + data_completa['DIA']
data_completa.rename(columns={'CMTC':'CMTS', 'COD_NODO':'NODO'}, inplace=True)
cols_stream = ['CMTS','CABLEMAC','NODO','OCUPACION','FECHA']
downstream = data_completa.loc[(data_completa['SENTIDO']=='DOWN'),cols_stream].copy()
downstream.rename(columns={'OCUPACION':'DW'}, inplace=True)
upstream = data_completa.loc[(data_completa['SENTIDO']=='UP'), cols_stream].copy()
upstream.rename(columns={'OCUPACION':'UP'}, inplace=True)

In [4]:
downstream.to_csv('Data_Forecast/dwstream.csv', sep=";",decimal=',') # Carga de datos
upstream.to_csv('Data_Forecast/upstream.csv', sep=";",decimal=',') # Carga de datos

In [5]:
limpiar_carpeta_out_modelo()

Verificar: si si


In [6]:
downstream['FECHA'].sort_values().unique()

array(['2020-06-01', '2020-06-08', '2020-06-15', '2020-06-22',
       '2020-06-29', '2020-07-06', '2020-07-13', '2020-07-20',
       '2020-07-27', '2020-08-03', '2020-08-10', '2020-08-17',
       '2020-08-24', '2020-08-31', '2020-09-07', '2020-09-14',
       '2020-09-21', '2020-09-28', '2020-10-05', '2020-10-12',
       '2020-10-19', '2020-10-26', '2020-11-02', '2020-11-09',
       '2020-11-16', '2020-11-23', '2020-11-30', '2020-12-07',
       '2020-12-14', '2020-12-21', '2020-12-28', '2021-01-04',
       '2021-01-11', '2021-01-18', '2021-01-25', '2021-02-01',
       '2021-02-08', '2021-02-15', '2021-02-22', '2021-03-01',
       '2021-03-08', '2021-03-15', '2021-03-22', '2021-03-29',
       '2021-04-05', '2021-04-12', '2021-04-19', '2021-04-26',
       '2021-05-03', '2021-05-10', '2021-05-17', '2021-05-24',
       '2021-05-31', '2021-06-07', '2021-06-14'], dtype=object)

In [7]:
#### fechas de lunes (automatizar fechas)
#### se recomienda dejar tres fechas de validación y predecir dos
fecha_ini_fore = '2021-05-31'
fecha_final_fore  = '2021-06-28'
# Para correr todos colocar -1
n_nodos = 100

In [8]:
run_modelo(downstream, "DW",n_nodos, 
           fecha_ini_fore, fecha_final_fore, load_model=False)

--------------------------------------------------
Tiempo de inicio de la corrida del modelo: 2021-07-15 15:11
--------------------------------------------------


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  6.1min finished


--------------------------------------------------
Tiempo de finalización de la corrida del modelo: 2021-07-15 15:17
--------------------------------------------------


In [9]:
run_modelo(upstream, "UP",n_nodos, 
           fecha_ini_fore, fecha_final_fore, load_model=False)

--------------------------------------------------
Tiempo de inicio de la corrida del modelo: 2021-07-15 15:17
--------------------------------------------------


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  5.3min finished


--------------------------------------------------
Tiempo de finalización de la corrida del modelo: 2021-07-15 15:22
--------------------------------------------------


In [10]:
df_evaluacion = evaluacion_mape_modelo()

100%|██████████| 179/179 [00:19<00:00,  9.16it/s]


En el nodo del archivo Data_out_modelo\DW_SGT4D.csv no se pudo calcular el mape, se asigna 2 como valor por defecto
************************************************************
Información de la métrica de evaluación (mape):
************************************************************
EXCELENTE (<=10%)    61.0
BUENO (<=20%)        27.0
MALO (>30%)           6.0
REGULAR (<=30%)       6.0
Name: CALIFICACION, dtype: float64
------------------------------------------------------------
Descripción de los puntajes clasificados como malos: 
------------------------------------------------------------
               MAPE
count  1.000000e+01
mean   1.891512e+09
std    5.981486e+09
min    3.024219e-01
25%    4.067106e-01
50%    4.713308e-01
75%    7.396388e-01
max    1.891512e+10
--------------------------------------------------
Tiempo de ejecución de la evaluación: 19.768937349319458
--------------------------------------------------
--------------------------------------------------
Evaluació

In [12]:
### Número de periodos a proyectar se puede automatizar como 2 teniendo en cuenta que las fechas indicadas anteriormente den dos periodos de proyección
periodos_a_proyectar = 2
consolidar_archivo_proyeccion(periodos_a_proyectar)

100%|██████████| 179/179 [00:00<00:00, 268.19it/s]


------------------------------------------------------------
Conteo de nodos: 
 DOWN:
93 
 UP:
86 
------------------------------------------------------------
Conteo de fechas (): 
 Down:
2021-06-28    93
2021-06-21    93
Name: FECHA, dtype: int64 
 UP:
2021-06-28    86
2021-06-21    86
Name: FECHA, dtype: int64
------------------------------------------------------------
Proyecciones guardadas en: Data_Forecast/proyecciones/


In [13]:
# consolidación de archivos para análisis temporal de tendencias 
# de los tráficos, ver notebook claro_nodos_temporal.ipynb
consolidar_archivo_serie_temporal()

100%|██████████| 179/179 [00:00<00:00, 185.56it/s]


----------------------------------------------------------------------------------------------------
Los siguientes archivos no se han agregado a los archivos de series temporales
----------------------------------------------------------------------------------------------------
['Data_out_modelo\\DW_SGT4D.csv']
------------------------------------------------------------
Conteo de nodos: 
 Serie real:
178 
 Proyecciones:
178 
------------------------------------------------------------
Conteo de fechas: 
 Reales:
2021-01-04    178
2021-04-26    178
2021-02-08    178
2021-02-01    178
2021-05-03    178
2021-01-25    178
2021-02-22    178
2021-04-05    178
2021-03-29    178
2021-06-07    178
2021-03-15    178
2021-05-31    178
2021-04-12    178
2021-03-08    178
2021-04-19    178
2021-02-15    178
2021-05-10    178
2021-03-01    178
2021-01-18    178
2021-03-22    178
2021-05-17    177
2020-12-07    177
2020-11-30    177
2020-12-28    177
2021-05-24    177
2020-12-14    177
2020-10-19 

In [14]:
downstream

,CMTS,CABLEMAC,NODO,DW,FECHA
1,BOGO-COT-H-03-CS100G,CATV-MAC 17,TN11,0.616279,2021-02-08
2,VILL-VILL-H-02-CS100G,CATV-MAC 5,VGL,0.672510,2021-06-07
6,BOGO-POLO-H-15-CS100G,CATV-MAC 19,30A013,0.473507,2021-02-01
7,PAST-PASTO-H-01-CS100G,CATV-MAC 37,NVR2D,0.584218,2020-07-13
9,BOGO-FONT-H-02-CS100G,CATV-MAC 13,41D1,0.653036,2020-12-14
...,...,...,...,...,...
1478833,BOGO-VENE-H-06-CS100G,CATV-MAC 10,GLC1,0.392465,2020-10-26
1478837,TUNJ-TUNJA-H-01-CS100G,CATV-MAC 13,SRN,0.387603,2020-11-02
1478838,MEDE-ESUR-H-03-ARRIE6,CABLE-MAC 54,ENG,0.627140,2021-02-22
1478839,CHIA-CHIA-H-03-CS100G,CATV-MAC 11,ILI2D,0.811810,2021-01-25


In [15]:
# fecha inicial corresponde a la penultima semana y final a la última
# semana de la que se tiene registro
divi = {'UP':(27*4), 'DW':(38*48)}
downstream['DW'] = downstream['DW']*divi['DW']
upstream['UP'] = upstream['UP']*divi['UP']
ocup_completo = generar_archivo_unificado(downstream, upstream)
ocup_wide = get_ocup_wide(ocup_completo)

Fechas:  Inicial --> 2021-06-07, Final --> 2021-06-14


In [16]:
# elimiar repetidos, se conserva el de valor máximo en down para 
# la última fecha
ocup_wide.sort_values('DW_FIN', ascending=False, inplace=True)
ocup_wide.drop_duplicates(subset=['NODO'], inplace=True)
sum(ocup_wide.duplicated(subset='NODO'))

0

#### MODIFICAR CADA VEZ QUE SE CAMBIE EL LISTADO DE REQUERIMIENTOS.

In [17]:
path_reque = 'Data_priorizacion\Requerimientos_blindaje\Listado Requerimientosxlsx.xlsx'
blindajes = pd.read_excel(path_reque, dtype = {'ID NODO':str, 'ID ALTERNO':str})

#### No modificar

In [18]:
pred_dw = pd.read_csv('Data_Forecast/proyecciones/dw_proyeccion.csv')
pred_up = pd.read_csv('Data_Forecast/proyecciones/up_proyeccion.csv')
pred_dw.drop_duplicates(subset=['NODO','FECHA'],inplace=True)
pred_up.drop_duplicates(subset=['NODO','FECHA'],inplace=True)
ocup_blindaje = ocup_wide.merge(blindajes, left_on = 'NODO', right_on ='ID NODO', how = 'left')
ocup_blindaje.loc[ocup_blindaje['NOMBRE DEL PLAN'].isna(), 'NOMBRE DEL PLAN'] = 'Resto del Pais'

#### MODIFICAR CADA VEZ QUE SE CORRA EL ANÁLISIS

In [19]:
# automatizar fechas de las semanas posteriores en las entradas de la siguiente función
ocup_proyeccion = get_ocup_proyeccion(pred_dw, pred_up, ocup_blindaje,'2021-06-21','2021-06-28')   

In [20]:
df_evaluacion = pd.read_csv('Data_Forecast/proyecciones/evaluacion.csv')
serie_real = pd.read_csv('Data_Forecast/series_temporales/series_reales.csv')
tabla_frec_rojos = get_frecuencia_rojos(serie_real)
priorizacion = get_priorizacion(ocup_proyeccion, 
                                df_evaluacion, tabla_frec_rojos)

Los csv de priorización fueron guardados en la carpeta Data_priorizacion.


***Para generar el archivo con información geográfica ir al cuadernillo análisis_geo***